In [ ]:
import pandas as pd
import time

def ask_with_id(self, query_id, query, context=None):
    """Method where you explicitly provide the query_id"""
    if context is None:
        context = "Each document represents a task. All tasks belong to a same workflow execution trace. "
        context += "The time the task started is stored in the started_at. The time the task ended is stored in the ended_at. The task duration is ended_at - started_at for each task "
    
    # Prepare full query text
    full_query = f"{context}. {query}"
    
    # Time the query
    t0 = time.time()
    result = self.qa_chain({"query": full_query})
    response_time = time.time() - t0
    
# GETTING RESPONSE TEXT
    response_text = result["result"]
    
# CHAR COUNTS
    query_chars = len(query)
    response_chars = len(response_text)
    
# ADDING TO DF
    new_row = {
        'Query_ID': query_id,
        'Query_Text': query,
        'Query_Chars': query_chars,
        'Response_Text': response_text,
        'Response_Chars': response_chars,
        'Response_Time': response_time,
        'Accuracy': None  # To be filled manually
    }
    
    self.query_df = pd.concat([self.query_df, pd.DataFrame([new_row])], ignore_index=True)
    
# PRINTING RESULTS
    print(f"Q: {query}")
    print(response_text)
    print(f"---------------- I took {response_time:.1f} s to answer this.")
    print("\n\n")
    
    return result

# ADDING METHOD
import types
qa.ask_with_id = types.MethodType(ask_with_id, qa)

In [ ]:
# Example usage:
"""
# Define your query suite
my_queries = [
    {
        "base_id": "DF-DL-Q01-NS-NC-R1-Swallow",
        "query": "What is the data lineage for this workflow?",
        "runs": 3,
        "use_cot": False,
        "context": None
    },
    {
        "base_id": "CF-EO-Q02-NS-NC-R5-Swallow",
        "query": "What is the execution order of tasks?",
        "runs": 3,
        "use_cot": False,
        "context": None
    },
    {
        "base_id": "TT-TDT-Q03-CoT-FC-R12-Swallow",
        "query": "What are the task durations?",
        "runs": 3,
        "use_cot": True,
        "context": "Full context with detailed timing information"
    }
]

# Run all queries
results = run_query_suite(qa, my_queries)

# Generate query IDs for batch accuracy updates
df_dl_ids = generate_query_ids_for_batch("DF-DL-Q01-NS-NC-Swallow", 3, 1)
add_batch_accuracy(qa, df_dl_ids, [0.85, 0.90, 0.88])

# Or use a single score for all runs
cf_eo_ids = generate_query_ids_for_batch("CF-EO-Q02-NS-NC-Swallow", 3, 5)
add_batch_accuracy(qa, cf_eo_ids, 0.92)

# View results
print(qa.query_df)

# Export results
qa.export_queries("automated_query_results.csv")
"""